In [18]:
# Veri Seti Hikayesi:
# Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

# Değişkenler:
# reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
# asin - ID of the product, e.g. 0000013714
# reviewerName - name of the reviewer
# helpful - helpfulness rating of the review, e.g. 2/3
# reviewText - text of the review
# overall - rating of the product
# summary - summary of the review
# unixReviewTime - time of the review (unix time)
# reviewTime - time of the review (raw)


In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st
import sys

In [20]:
#Görev 1: Average Rating'i Güncel Yorumlara Göre Hesaplamak ve Var Olan Average Rating ile Kıyaslamak.

# Paylaşılan veri setinde kullanıcılar bir ürüne puanlar vermiş ve yorumlar yapmıştır.
# Bu görevde amacımız verilen puanları tarihe göre ağırlıklandırarak değerlendirmek.
# İlk ortalama puan ile elde edilecek tarihe göre ağırlıklı puanın karşılaştırılması gerekmektedir.




In [21]:
#Ürünün ortalama puanı

df = pd.read_csv("amazon_review.csv")
df["overall"].mean()

4.587589013224822

In [22]:
# Son zamanlardaki trendin kaçırılma durumu var. Örneğin Ürünle iligili kullanıldıktan sonra çeşitli arzaların ortaya çıkması gibi.
# Bu durumda tarihe göre bir ağırlıklandırma yapıp yeni bir rating hesaplanabilir. Böylece son zamanlardaki pozitif trend veya
# negait trend durumu yakalanabilir duruma geliyor olacakatır.

In [23]:
# Tarihe göre ağırlıklandırma

# Tarihlere göre ağırlıklı puan hesabı yapabilmek için:
#   - reviewTime değişkenini tarih değişkeni olarak tanıtmanız
#   - reviewTime'ın max değerini current_date olarak kabul etmeniz
#   - her bir puan-yorum tarihi ile current_date'in farkını gün cinsinden ifade ederek yeni değişken oluşturmanız
#   - ve gün cinsinden ifade edilen değişkeni quantile fonksiyonu ile 4'e bölüp (3 çeyrek verilirse 4 parça çıkar)
#   - çeyrekliklerden gelen değerlere göre ağırlıklandırma yapmanız gerekir.
#   - örneğin q1 = 12 ise ağırlıklandırırken 12 günden az süre önce yapılan yorumların ortalamasını alıp bunlara
#   - yüksek ağırlık vermek gibi.

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [25]:
# day_diff: yorum sonrası ne kadar gün geçmiş
df['reviewTime'] = pd.to_datetime(df['reviewTime'], dayfirst=True)
current_date = pd.to_datetime(str(df['reviewTime'].max()))
df["day_diff"] = (current_date - df['reviewTime']).dt.days

In [26]:
# day_diff değişkeninin çeyrek değerlerini elde etme:
a = df["day_diff"].quantile(0.25)
b = df["day_diff"].quantile(0.50)
c = df["day_diff"].quantile(0.75)

In [27]:
# a,b,c değerlerine göre ağırlıklı puanı hesaplama:
df.loc[df["day_diff"] <= a, "overall"].mean() * 30 / 100 + \
df.loc[(df["day_diff"] > a) & (df["day_diff"] <= b), "overall"].mean() * 26 / 100 + \
df.loc[(df["day_diff"] > b) & (df["day_diff"] <= c), "overall"].mean() * 24 / 100 + \
df.loc[(df["day_diff"] > c), "overall"].mean() * 20 / 100

4.600583941300071

In [28]:
# Görev 2: Ürün için Ürün Detay Sayfasında Görüntülenecek 20 Review'i Belirleyiniz.
# Adım 1. helpful_no Değişkenini Üretiniz
###################################################

# Not:
# total_vote bir yoruma verilen toplam up-down sayısıdır.
# up, helpful demektir.
# veri setinde helpful_no değişkeni yoktur, var olan değişkenler üzerinden üretilmesi gerekmektedir.

df["helpful_no"] = df["total_vote"] - df["helpful_yes"]
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.0,Four Stars,0,0,0,2014-07-23
1,0mie,5.0,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.0,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.0,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.0,best deal around,0,0,0,2013-07-13


In [29]:
# Adım 2. score_pos_neg_diff, score_average_rating ve wilson_lower_bound Skorlarını Hesaplayıp Veriye Ekleyiniz

def score_up_down_diff(up, down):
    return up - down

# score_pos_neg_diff
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

df.sort_values("score_pos_neg_diff", ascending=False).head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.0,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.0,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.0,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
4596,"Tom Henriksen ""Doggy Diner""",1.0,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55
1835,goconfigure,5.0,I own it,60,8,68,2014-02-28,52
4672,Twister,5.0,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41
4306,Stellar Eller,5.0,Awesome Card!,51,14,65,2012-09-06,37
315,"Amazon Customer ""johncrea""",5.0,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28


In [30]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)


# score_average_rating
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

df.sort_values("score_average_rating", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.0,Perfect!!,1,0,1,2012-12-19,1,1.0
2881,Lou Thomas,5.0,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.0
1073,C. Sanchez,5.0,Tons of space for phone,1,0,1,2013-08-13,1,1.0
445,"Apache ""Elizabeth""",4.0,Amazon Great Prices,1,0,1,2013-12-18,1,1.0
3923,Rock Your Roots,5.0,What more to say?,1,0,1,2013-12-30,1,1.0
435,Anthony L cate,5.0,Love the extra storage,1,0,1,2012-07-24,1,1.0
2901,luis,5.0,Awesome and fast card :),1,0,1,2013-05-13,1,1.0
2204,"jbwam ""jbwam""",2.0,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.0
2206,JCBiker,5.0,Great card,1,0,1,2013-10-31,1,1.0
3408,"Neng Vang ""Neng2012""",5.0,working no problem,1,0,1,2013-07-25,1,1.0


In [31]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)



df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.966337,0.957544
3449,NLee the Engineer,5.0,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.948837,0.936519
4212,SkincareCEO,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1.0,"Warning, read this!",422,73,495,2012-02-09,349,0.852525,0.818577
4672,Twister,5.0,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.918367,0.808109
1835,goconfigure,5.0,I own it,60,8,68,2014-02-28,52,0.882353,0.784651
3981,"R. Sutton, Jr. ""RWSynergy""",5.0,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.805755,0.732136
3807,R. Heisler,3.0,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.880000,0.700442
4306,Stellar Eller,5.0,Awesome Card!,51,14,65,2012-09-06,37,0.784615,0.670334
4596,"Tom Henriksen ""Doggy Diner""",1.0,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.752294,0.663595
